In [7]:
import dropbox
import shutil
import os

DOWNLOAD_FOLDER = "content"
TEST_CONTENT_DROPBOX_FOLDER = "LLM Doc Exp Test Content"
API_KEY = os.environ.get('DROPBOX_API_KEY', "DUMMY_KEY")
dbx = dropbox.Dropbox(API_KEY)


In [8]:
# Ensure local download folder exists, and delete its contents

def create_download_folder():
    if not os.path.exists(DOWNLOAD_FOLDER):
        os.makedirs(DOWNLOAD_FOLDER)

def clear_downloads_folder():
    for filename in os.listdir(DOWNLOAD_FOLDER):
        file_path = os.path.join(DOWNLOAD_FOLDER, filename)
        print(file_path)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

create_download_folder()
clear_downloads_folder()

content/Quick Writes Technical Documentation.md
content/Rails Migrations Guide.md
content/Guided Drafts Technical Documentation.md
content/Writing Assignment Data Documentation.md


In [9]:
# Filter dropbox paper docs and download to content file
from dropbox.paper import ExportFormat, ListPaperDocsFilterBy

def get_file_path(doc_id):
    return os.path.join(DOWNLOAD_FOLDER, f"{doc_titles[doc_id]}.md")

def download_doc(doc_id):
    result = dbx.paper_docs_download_to_file(get_file_path(doc_id), doc_id, ExportFormat('markdown'))
    print(f"- downloaded '{result.title}'")
    return result

print("Retrieving document IDs")
doc_ids = dbx.paper_docs_list(filter_by=ListPaperDocsFilterBy.docs_created).doc_ids
print(f"- {len(doc_ids)} documents found")

print("Filtering documents in folder")
docs_ids_in_folder = [doc_id for doc_id in doc_ids if TEST_CONTENT_DROPBOX_FOLDER in [folder.name for folder in dbx.paper_docs_get_folder_info(doc_id).folders or []]]
print(f"- {len(docs_ids_in_folder)} documents found in folder")

print("Retrieving document titles")
doc_titles = {doc_id: dbx.paper_docs_download(doc_id, ExportFormat('markdown'))[0].title for doc_id in docs_ids_in_folder}

print("Downloading documents")
results = [download_doc(doc_id) for doc_id in docs_ids_in_folder]
print("Download complete")

Retrieving document IDs
- 92 documents found
Filtering documents in folder
- 29 documents found in folder
Retrieving document titles
- downloaded 'Immigrate to Nunavut'
- downloaded 'Quebec Immigration programs for skilled workers'
- downloaded 'About The BC Provincial Nominee Program'
- downloaded 'Eligibility to apply for the Federal Skilled Trades Program (Express Entry)'
- downloaded 'Eligibility to apply as a Federal Skilled Worker (Express Entry)'
- downloaded 'How Express Entry works'
- downloaded 'Immigrate as a provincial nominee'
- downloaded 'Find your National Occupation Classification (NOC)'
- downloaded 'Offer of employment – Skilled immigrants (Express Entry)'
- downloaded 'LMIA-exempt job offers – skilled immigrants (Express Entry)'
- downloaded 'Language testing—Skilled immigrants (Express Entry)'
- downloaded 'Language requirements — Skilled immigrants (Express Entry)'
- downloaded 'Educational credential assessment (ECA) for Express Entry'
- downloaded 'I am working 